In [ ]:
import optuna
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier


# ==========================
# Random Forest
# ==========================
def objective_rf(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False])
    }
    model = RandomForestClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


# ==========================
# XGBoost
# ==========================
def objective_xgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10)
    }
    model = xgb.XGBClassifier(
        **params, random_state=42, use_label_encoder=False, eval_metric="logloss"
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


# ==========================
# LightGBM
# ==========================
def objective_lgb(trial):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 200),
        "max_depth": trial.suggest_int("max_depth", -1, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 10),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 10)
    }
    model = lgb.LGBMClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


# ==========================
# CatBoost
# ==========================
def objective_cat(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "border_count": trial.suggest_int("border_count", 32, 255)
    }
    model = CatBoostClassifier(
        **params, random_state=42, verbose=0
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


# ==========================
# KNN
# ==========================
def objective_knn(trial):
    params = {
        "n_neighbors": trial.suggest_int("n_neighbors", 1, 50),
        "weights": trial.suggest_categorical("weights", ["uniform", "distance"]),
        "p": trial.suggest_int("p", 1, 2)
    }
    model = KNeighborsClassifier(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


# ==========================
# Logistic Regression
# ==========================
def objective_logreg(trial):
    params = {
        "penalty": trial.suggest_categorical("penalty", ["l1", "l2", "elasticnet", "none"]),
        "C": trial.suggest_float("C", 0.001, 10, log=True),
        "solver": trial.suggest_categorical("solver", ["liblinear", "saga", "lbfgs"]),
    }
    # elasticnet requires l1_ratio
    if params["penalty"] == "elasticnet":
        params["l1_ratio"] = trial.suggest_float("l1_ratio", 0, 1)
    model = LogisticRegression(max_iter=2000, random_state=42, **params)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


# ==========================
# Decision Tree
# ==========================
def objective_dt(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    }
    model = DecisionTreeClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_rf, n_trials=50)
print("Best RF params:", study.best_params)


In [ ]:
import optuna
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor


# ==========================
# Random Forest Regressor
# ==========================
def objective_rf_reg(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False])
    }
    model = RandomForestRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


# ==========================
# XGBoost Regressor
# ==========================
def objective_xgb_reg(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10)
    }
    model = xgb.XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


# ==========================
# LightGBM Regressor
# ==========================
def objective_lgb_reg(trial):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 200),
        "max_depth": trial.suggest_int("max_depth", -1, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 10),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 10)
    }
    model = lgb.LGBMRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


# ==========================
# CatBoost Regressor
# ==========================
def objective_cat_reg(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "border_count": trial.suggest_int("border_count", 32, 255)
    }
    model = CatBoostRegressor(**params, random_state=42, verbose=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


# ==========================
# KNN Regressor
# ==========================
def objective_knn_reg(trial):
    params = {
        "n_neighbors": trial.suggest_int("n_neighbors", 1, 50),
        "weights": trial.suggest_categorical("weights", ["uniform", "distance"]),
        "p": trial.suggest_int("p", 1, 2)
    }
    model = KNeighborsRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)

# ==========================
# Decision Tree Regressor
# ==========================
def objective_dt_reg(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    }
    model = DecisionTreeRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)



# ==========================
# Linear Regression
# ==========================
def objective_lin_reg(trial):
    model = LinearRegression()
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


# ==========================
# Ridge Regression
# ==========================
def objective_ridge(trial):
    params = {
        "alpha": trial.suggest_float("alpha", 0.0001, 10, log=True)
    }
    model = Ridge(**params, random_state=42, max_iter=2000)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


# ==========================
# Lasso Regression
# ==========================
def objective_lasso(trial):
    params = {
        "alpha": trial.suggest_float("alpha", 0.0001, 10, log=True)
    }
    model = Lasso(**params, random_state=42, max_iter=2000)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


# ==========================
# ElasticNet Regression
# ==========================
def objective_enet(trial):
    params = {
        "alpha": trial.suggest_float("alpha", 0.0001, 10, log=True),
        "l1_ratio": trial.suggest_float("l1_ratio", 0, 1)
    }
    model = ElasticNet(**params, random_state=42, max_iter=2000)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


# ==========================
# Polynomial Regression
# ==========================
def objective_poly(trial):
    degree = trial.suggest_int("degree", 2, 5)
    model = Pipeline([
        ("poly", PolynomialFeatures(degree=degree)),
        ("lin_reg", LinearRegression())
    ])
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_rf_reg, n_trials=50)
print("Best RF Regressor params:", study.best_params)


# Unified Classification Objective

In [ ]:
def objective_classification(trial):
    model_name = trial.suggest_categorical("model", ["RandomForest", "DecisionTree", "XGBoost", 
                                                     "LightGBM", "CatBoost", "KNN", "LogisticRegression"])
    
    # -----------------------------
    # Random Forest
    # -----------------------------
    if model_name == "RandomForest":
        params = {
            "n_estimators": trial.suggest_int("rf_n_estimators", 100, 1000),
            "max_depth": trial.suggest_int("rf_max_depth", 3, 30),
            "min_samples_split": trial.suggest_int("rf_min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("rf_min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("rf_max_features", ["sqrt", "log2", None]),
            "bootstrap": trial.suggest_categorical("rf_bootstrap", [True, False])
        }
        model = RandomForestClassifier(**params, random_state=42)
    
    # -----------------------------
    # Decision Tree
    # -----------------------------
    elif model_name == "DecisionTree":
        params = {
            "max_depth": trial.suggest_int("dt_max_depth", 3, 30),
            "min_samples_split": trial.suggest_int("dt_min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("dt_min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("dt_max_features", ["sqrt", "log2", None])
        }
        model = DecisionTreeClassifier(**params, random_state=42)
    
    # -----------------------------
    # XGBoost
    # -----------------------------
    elif model_name == "XGBoost":
        params = {
            "n_estimators": trial.suggest_int("xgb_n_estimators", 100, 1000),
            "learning_rate": trial.suggest_float("xgb_learning_rate", 0.01, 0.3),
            "max_depth": trial.suggest_int("xgb_max_depth", 3, 15),
            "subsample": trial.suggest_float("xgb_subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("xgb_colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("xgb_gamma", 0, 10),
            "min_child_weight": trial.suggest_int("xgb_min_child_weight", 1, 10)
        }
        model = xgb.XGBClassifier(**params, random_state=42, use_label_encoder=False, eval_metric="logloss")
    
    # -----------------------------
    # LightGBM
    # -----------------------------
    elif model_name == "LightGBM":
        params = {
            "num_leaves": trial.suggest_int("lgb_num_leaves", 20, 200),
            "max_depth": trial.suggest_int("lgb_max_depth", -1, 20),
            "learning_rate": trial.suggest_float("lgb_learning_rate", 0.01, 0.3),
            "n_estimators": trial.suggest_int("lgb_n_estimators", 100, 1000),
            "min_child_samples": trial.suggest_int("lgb_min_child_samples", 5, 50),
            "subsample": trial.suggest_float("lgb_subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("lgb_colsample_bytree", 0.5, 1.0),
            "reg_alpha": trial.suggest_float("lgb_reg_alpha", 0, 10),
            "reg_lambda": trial.suggest_float("lgb_reg_lambda", 0, 10)
        }
        model = lgb.LGBMClassifier(**params, random_state=42)
    
    # -----------------------------
    # CatBoost
    # -----------------------------
    elif model_name == "CatBoost":
        params = {
            "iterations": trial.suggest_int("cat_iterations", 100, 1000),
            "depth": trial.suggest_int("cat_depth", 3, 12),
            "learning_rate": trial.suggest_float("cat_learning_rate", 0.01, 0.3),
            "l2_leaf_reg": trial.suggest_float("cat_l2_leaf_reg", 1, 10),
            "border_count": trial.suggest_int("cat_border_count", 32, 255)
        }
        model = CatBoostClassifier(**params, random_state=42, verbose=0)
    
    # -----------------------------
    # KNN
    # -----------------------------
    elif model_name == "KNN":
        params = {
            "n_neighbors": trial.suggest_int("knn_n_neighbors", 1, 50),
            "weights": trial.suggest_categorical("knn_weights", ["uniform", "distance"]),
            "p": trial.suggest_int("knn_p", 1, 2)
        }
        model = KNeighborsClassifier(**params)
    
    # -----------------------------
    # Logistic Regression
    # -----------------------------
    elif model_name == "LogisticRegression":
        params = {
            "penalty": trial.suggest_categorical("log_penalty", ["l1", "l2", "elasticnet", "none"]),
            "C": trial.suggest_float("log_C", 0.001, 10, log=True),
            "solver": trial.suggest_categorical("log_solver", ["liblinear", "saga", "lbfgs"])
        }
        if params["penalty"] == "elasticnet":
            params["l1_ratio"] = trial.suggest_float("log_l1_ratio", 0, 1)
        model = LogisticRegression(max_iter=2000, random_state=42, **params)
    
    # -----------------------------
    # Fit & Evaluate
    # -----------------------------
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


# Unified Regression Objective

In [ ]:
def objective_regression(trial):
    model_name = trial.suggest_categorical("model", ["RandomForest", "DecisionTree", "XGBoost", 
                                                     "LightGBM", "CatBoost", "KNN", 
                                                     "Linear", "Ridge", "Lasso", "ElasticNet", "Polynomial"])
    
    # -----------------------------
    # Random Forest
    # -----------------------------
    if model_name == "RandomForest":
        params = {
            "n_estimators": trial.suggest_int("rf_n_estimators", 100, 1000),
            "max_depth": trial.suggest_int("rf_max_depth", 3, 30),
            "min_samples_split": trial.suggest_int("rf_min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("rf_min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("rf_max_features", ["sqrt", "log2", None]),
            "bootstrap": trial.suggest_categorical("rf_bootstrap", [True, False])
        }
        model = RandomForestRegressor(**params, random_state=42)
    
    # -----------------------------
    # Decision Tree
    # -----------------------------
    elif model_name == "DecisionTree":
        params = {
            "max_depth": trial.suggest_int("dt_max_depth", 3, 30),
            "min_samples_split": trial.suggest_int("dt_min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("dt_min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("dt_max_features", ["sqrt", "log2", None])
        }
        model = DecisionTreeRegressor(**params, random_state=42)
    
    # -----------------------------
    # XGBoost
    # -----------------------------
    elif model_name == "XGBoost":
        params = {
            "n_estimators": trial.suggest_int("xgb_n_estimators", 100, 1000),
            "learning_rate": trial.suggest_float("xgb_learning_rate", 0.01, 0.3),
            "max_depth": trial.suggest_int("xgb_max_depth", 3, 15),
            "subsample": trial.suggest_float("xgb_subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("xgb_colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("xgb_gamma", 0, 10),
            "min_child_weight": trial.suggest_int("xgb_min_child_weight", 1, 10)
        }
        model = xgb.XGBRegressor(**params, random_state=42)
    
    # -----------------------------
    # LightGBM
    # -----------------------------
    elif model_name == "LightGBM":
        params = {
            "num_leaves": trial.suggest_int("lgb_num_leaves", 20, 200),
            "max_depth": trial.suggest_int("lgb_max_depth", -1, 20),
            "learning_rate": trial.suggest_float("lgb_learning_rate", 0.01, 0.3),
            "n_estimators": trial.suggest_int("lgb_n_estimators", 100, 1000),
            "min_child_samples": trial.suggest_int("lgb_min_child_samples", 5, 50),
            "subsample": trial.suggest_float("lgb_subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("lgb_colsample_bytree", 0.5, 1.0),
            "reg_alpha": trial.suggest_float("lgb_reg_alpha", 0, 10),
            "reg_lambda": trial.suggest_float("lgb_reg_lambda", 0, 10)
        }
        model = lgb.LGBMRegressor(**params, random_state=42)
    
    # -----------------------------
    # CatBoost
    # -----------------------------
    elif model_name == "CatBoost":
        params = {
            "iterations": trial.suggest_int("cat_iterations", 100, 1000),
            "depth": trial.suggest_int("cat_depth", 3, 12),
            "learning_rate": trial.suggest_float("cat_learning_rate", 0.01, 0.3),
            "l2_leaf_reg": trial.suggest_float("cat_l2_leaf_reg", 1, 10),
            "border_count": trial.suggest_int("cat_border_count", 32, 255)
        }
        model = CatBoostRegressor(**params, random_state=42, verbose=0)
    
    # -----------------------------
    # KNN
    # -----------------------------
    elif model_name == "KNN":
        params = {
            "n_neighbors": trial.suggest_int("knn_n_neighbors", 1, 50),
            "weights": trial.suggest_categorical("knn_weights", ["uniform", "distance"]),
            "p": trial.suggest_int("knn_p", 1, 2)
        }
        model = KNeighborsRegressor(**params)
    
    # -----------------------------
    # Linear Regression
    # -----------------------------
    elif model_name == "Linear":
        model = LinearRegression()
    
    # -----------------------------
    # Ridge
    # -----------------------------
    elif model_name == "Ridge":
        alpha = trial.suggest_float("ridge_alpha", 0.0001, 10, log=True)
        model = Ridge(alpha=alpha, random_state=42, max_iter=2000)
    
    # -----------------------------
    # Lasso
    # -----------------------------
    elif model_name == "Lasso":
        alpha = trial.suggest_float("lasso_alpha", 0.0001, 10, log=True)
        model = Lasso(alpha=alpha, random_state=42, max_iter=2000)
    
    # -----------------------------
    # ElasticNet
    # -----------------------------
    elif model_name == "ElasticNet":
        alpha = trial.suggest_float("enet_alpha", 0.0001, 10, log=True)
        l1_ratio = trial.suggest_float("enet_l1_ratio", 0, 1)
        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42, max_iter=2000)
    
    # -----------------------------
    # Polynomial Regression
    # -----------------------------
    elif model_name == "Polynomial":
        degree = trial.suggest_int("poly_degree", 2, 5)
        model = Pipeline([
            ("poly", PolynomialFeatures(degree=degree)),
            ("lin_reg", LinearRegression())
        ])
    
    # -----------------------------
    # Fit & Evaluate
    # -----------------------------
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)


# Unified Classification Objective (Essential Hyperparameters)

In [ ]:
def objective_classification(trial):
    model_name = trial.suggest_categorical("model", ["RandomForest", "DecisionTree", "XGBoost", 
                                                     "LightGBM", "CatBoost", "KNN", "LogisticRegression"])
    
    if model_name == "RandomForest":
        model = RandomForestClassifier(
            n_estimators=trial.suggest_int("rf_n_estimators", 100, 200),
            max_depth=trial.suggest_int("rf_max_depth", 5, 15),
            random_state=42
        )
    
    elif model_name == "DecisionTree":
        model = DecisionTreeClassifier(
            max_depth=trial.suggest_int("dt_max_depth", 5, 15),
            min_samples_leaf=trial.suggest_int("dt_min_samples_leaf", 1, 5),
            criterion=trial.suggest_categorical("dt_criterion", ["gini", "entropy"]),
            random_state=42
        )
    
    elif model_name == "XGBoost":
        model = xgb.XGBClassifier(
            n_estimators=trial.suggest_int("xgb_n_estimators", 100, 200),
            max_depth=trial.suggest_int("xgb_max_depth", 3, 7),
            learning_rate=trial.suggest_float("xgb_learning_rate", 0.05, 0.2),
            random_state=42,
            use_label_encoder=False,
            eval_metric="logloss"
        )
    
    elif model_name == "LightGBM":
        model = lgb.LGBMClassifier(
            n_estimators=trial.suggest_int("lgb_n_estimators", 100, 200),
            max_depth=trial.suggest_int("lgb_max_depth", 3, 10),
            num_leaves=trial.suggest_int("lgb_num_leaves", 20, 50),
            learning_rate=trial.suggest_float("lgb_learning_rate", 0.05, 0.15),
            random_state=42
        )
    
    elif model_name == "CatBoost":
        model = CatBoostClassifier(
            iterations=trial.suggest_int("cat_iterations", 100, 200),
            depth=trial.suggest_int("cat_depth", 3, 6),
            learning_rate=trial.suggest_float("cat_learning_rate", 0.05, 0.2),
            random_state=42,
            verbose=0
        )
    
    elif model_name == "KNN":
        model = KNeighborsClassifier(
            n_neighbors=trial.suggest_int("knn_n_neighbors", 3, 15)
        )
    
    elif model_name == "LogisticRegression":
        model = LogisticRegression(
            C=trial.suggest_float("log_C", 0.1, 1.0),
            penalty="l2",
            solver="lbfgs",
            max_iter=2000,
            random_state=42
        )
    
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)


# Unified Regression Objective (Essential Hyperparameters)

In [ ]:
def objective_regression(trial):
    model_name = trial.suggest_categorical("model", ["RandomForest", "DecisionTree", "XGBoost", 
                                                     "LightGBM", "CatBoost", "KNN", 
                                                     "Linear", "Ridge", "Lasso", "ElasticNet", "Polynomial"])
    
    if model_name == "RandomForest":
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int("rf_n_estimators", 100, 200),
            max_depth=trial.suggest_int("rf_max_depth", 5, 15),
            random_state=42
        )
    
    elif model_name == "DecisionTree":
        model = DecisionTreeRegressor(
            max_depth=trial.suggest_int("dt_max_depth", 5, 15),
            min_samples_leaf=trial.suggest_int("dt_min_samples_leaf", 1, 5),
            criterion=trial.suggest_categorical("dt_criterion", ["squared_error", "friedman_mse", "absolute_error"]),
            random_state=42
        )

    
    elif model_name == "XGBoost":
        model = xgb.XGBRegressor(
            n_estimators=trial.suggest_int("xgb_n_estimators", 100, 200),
            max_depth=trial.suggest_int("xgb_max_depth", 3, 7),
            learning_rate=trial.suggest_float("xgb_learning_rate", 0.05, 0.2),
            random_state=42
        )
    
    elif model_name == "LightGBM":
        model = lgb.LGBMRegressor(
            n_estimators=trial.suggest_int("lgb_n_estimators", 100, 200),
            max_depth=trial.suggest_int("lgb_max_depth", 3, 10),
            num_leaves=trial.suggest_int("lgb_num_leaves", 20, 50),
            learning_rate=trial.suggest_float("lgb_learning_rate", 0.05, 0.15),
            random_state=42
        )
    
    elif model_name == "CatBoost":
        model = CatBoostRegressor(
            iterations=trial.suggest_int("cat_iterations", 100, 200),
            depth=trial.suggest_int("cat_depth", 3, 6),
            learning_rate=trial.suggest_float("cat_learning_rate", 0.05, 0.2),
            random_state=42,
            verbose=0
        )
    
    elif model_name == "KNN":
        model = KNeighborsRegressor(
            n_neighbors=trial.suggest_int("knn_n_neighbors", 3, 15)
        )
    
    elif model_name == "Linear":
        model = LinearRegression()
    
    elif model_name == "Ridge":
        model = Ridge(
            alpha=trial.suggest_float("ridge_alpha", 0.1, 1.0),
            random_state=42,
            max_iter=2000
        )
    
    elif model_name == "Lasso":
        model = Lasso(
            alpha=trial.suggest_float("lasso_alpha", 0.1, 1.0),
            random_state=42,
            max_iter=2000
        )
    
    elif model_name == "ElasticNet":
        model = ElasticNet(
            alpha=trial.suggest_float("enet_alpha", 0.1, 1.0),
            l1_ratio=trial.suggest_float("enet_l1_ratio", 0, 1),
            random_state=42,
            max_iter=2000
        )
    
    elif model_name == "Polynomial":
        degree = trial.suggest_int("poly_degree", 2, 3)
        model = Pipeline([
            ("poly", PolynomialFeatures(degree=degree)),
            ("lin_reg", LinearRegression())
        ])
    
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)
